In [4]:
# 모듈 임포트
import time
import board
import busio
import adafruit_ads1x15.ads1115 as ADS
from adafruit_ads1x15.analog_in import AnalogIn
import tensorflow as tf
import joblib
import numpy as np
import psutil

NotImplementedError: 
        Adafruit-PlatformDetect version 3.77.0 was unable to identify the board and/or
        microcontroller running the Windows platform. Please be sure you
        have the latest packages by running:
        'pip3 install --upgrade adafruit-blinka adafruit-platformdetect'

        If you are running the latest package, your board may not yet be supported. Please
        open a New Issue on GitHub at https://github.com/adafruit/Adafruit_Blinka/issues and
        select New Board Request.
        

In [ ]:
# 모델, 스케일러 로드
model = tf.keras.models.load_model('./model/cnn1d_model.h5')
scaler = joblib.load('./model/scaler_cnn.joblib')


In [ ]:
# I2C 및 ADC 설정
i2c = busio.I2C(board.SCL, board.SDA)
ads = ADS.ADS1115(i2c)
ads.gain = 1

voltage_ch = AnalogIn(ads, ADS.P0)
current_ch = AnalogIn(ads, ADS.P1)

In [ ]:
# 샘플 버퍼 및 이상 탐지 버퍼
data_buffer = []
anomaly_buffer = []
ANOMALY_THRESHOLD = 8
first_anomaly_detected = False

# 피처 생성 함수
def add_features(voltage_seq, current_seq):
    voltage = np.mean(voltage_seq) # 평균
    current = np.mean(current_seq) # 평균
    voltage_diff = np.abs(voltage_seq[-1] - voltage_seq[-2])
    current_diff = np.abs(current_seq[-1] - current_seq[-2])
    voltage_ma = np.mean(voltage_seq[-5:])
    current_ma = np.mean(current_seq[-5:])
    power = voltage * current
    power_diff = np.abs(power - (voltage_seq[-2] * current_seq[-2]))
    return [voltage, current, voltage_diff, current_diff, voltage_ma, current_ma, power, power_diff]


In [ ]:
# 실시간 센서 읽기 및 예측
try:
    print("🚀 실시간 아크 감지 시작합니다!")

    start_time = time.perf_counter()

    while True:
        voltage = voltage_ch.voltage
        current = current_ch.voltage

        data_buffer.append((voltage, current))

        if len(data_buffer) >= 8:
            voltage_seq, current_seq = zip(*data_buffer[-8:])

            features = add_features(voltage_seq, current_seq)
            features_scaled = scaler.transform(np.array(features).reshape(1, -1))
            features_scaled = features_scaled.reshape((1, len(features), 1))

            y_pred = model.predict(features_scaled, verbose=0)
            anomaly = int(y_pred.flatten() > 0.5)

            anomaly_buffer.append(anomaly)
            if len(anomaly_buffer) > ANOMALY_THRESHOLD:
                anomaly_buffer.pop(0)

            if sum(anomaly_buffer) >= ANOMALY_THRESHOLD:
                print("⚡ 아크 이상 감지!")
                if not first_anomaly_detected:
                    elapsed_time = time.perf_counter() - start_time
                    print(f"⏱️ 첫 번째 이상 탐지까지 걸린 시간: {elapsed_time:.6f}초")
                    first_anomaly_detected = True

        # 리소스 사용량 실시간 출력
        cpu_usage = psutil.cpu_percent(interval=0.0)
        memory_usage = psutil.Process().memory_info().rss / 1024 ** 2
        print(f"🧠 메모리 사용량: {memory_usage:.2f} MB, 🧮 CPU 사용률: {cpu_usage:.2f}%")

        time.sleep(0.0001)  # 10kHz 샘플링

except KeyboardInterrupt:
    print("🛑 실시간 감지를 종료합니다.")
    pass